# Ejemplo de regresión Heckman - Modelo de selección de muestra

## 1. Importación de bibliotecas y datos

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.iolib.summary2 import summary_col
import heckman as heckman

# Omiting WARNINGS
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read data
df = pd.read_stata('womenwk.dta')
df

,c1,c2,u,v,county,age,education,married,children,select,wagefull,wage
0,-0.436205,-0.096918,-0.218103,-0.375727,1.0,22,10,1,0,16.791273,12.782769,NaN
1,0.352141,0.300476,0.176070,0.461234,2.0,36,10,1,0,32.434814,20.312845,20.312845
2,1.077247,-1.595963,0.538623,-0.376244,3.0,28,10,1,0,19.185072,23.063478,NaN
3,1.021283,-1.710498,0.510642,-0.496999,4.0,37,10,1,0,21.336010,24.527700,NaN
4,-0.442960,0.308340,-0.221480,-0.092511,5.0,39,10,1,1,31.989870,16.142241,16.142241
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.322928,-0.402405,-0.161464,-0.513113,6.0,35,20,1,1,32.842648,26.062429,26.062429
1996,0.215594,0.953160,0.107797,0.828961,7.0,34,12,1,1,42.547531,21.093561,21.093561
1997,-1.967921,0.971543,-0.983961,-0.694023,8.0,36,16,1,5,48.071724,12.392473,12.392473
1998,-0.008480,-0.180160,-0.004240,-0.133984,9.0,48,12,1,0,31.592197,22.549122,22.549122


El modelo de Heckman asume que el salario (wage) es una variable dependendiente que es determinada primero por un modelo binario.

Por su parte, las variables married, children, educ y age se asumen como determinantes de la probabilidad de observar el salario. 

Así, el modelo será un probit para:
$$wage_i^* = \gamma_0 + \gamma_1 married + \gamma_2 children + \gamma_3 educ + \gamma_4 age + v_i$$

Asumimos que si el salario es observado, entonces la ecuación será:
$$wage_i = \beta_0 + \beta_1 educ_i + \beta_2 age_i + u_i$$

Donde $wage^*_i$ toma valores de 1 o 0, y where $u_i$ and $v_i$ have correlation $\rho$. En resumen, asumiremos que el salario por hora es una función de la educación y la edad, mientras que la probabilidad de trabajar (la probabilidad de que se observe el salario) es una función del estado civil, el número de hijos en el hogar y (implícitamente) el salario ( a través de la inclusión de la edad y la educación, que creemos que determina el salario).

## 2. Generación y transformación de datos

In [3]:
#Variables
Y = df.wage
W = df[['married', 'children', 'education', 'age']]
W = sm.add_constant(W)

X = df[['education', 'age']]
X = sm.add_constant(X)

## 3. Estimación

In [4]:
# En dos etapas
res = heckman.Heckman(Y, W, X).fit(method = 'twostep')
print(res.summary())

       Heckman Regression Results      
Dep. Variable:                     wage
Model:                          Heckman
Method:                Heckman Two-Step
Date:                  Wed, 17 May 2023
Time:                          07:59:19
No. Total Obs.:                    2000
No. Censored Obs.:                  657
No. Uncensored Obs.:               1343
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.9204     11.130     -1.161      0.246     -34.734       8.893
married       -0.4304      0.323     -1.333      0.182      -1.063       0.202
children      -0.6705      0.105     -6.358      0.000      -0.877      -0.464
education      1.2457      0.215      5.801      0.000       0.825       1.667
age            0.3971      0.135      2.940      0.003       0.132       0.662
                 coef    std err          z      P>|z|      [0.025      0.975]
--------

In [5]:
#
Y = df[df.wage > 0].wage
X = df[df.wage > 0][['education', 'age']]
X = sm.add_constant(X)

In [6]:
est = sm.OLS(Y, X).fit()
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     227.5
Date:                Wed, 17 May 2023   Prob (F-statistic):           8.72e-86
Time:                        08:03:40   Log-Likelihood:                -4181.8
No. Observations:                1343   AIC:                             8370.
Df Residuals:                    1340   BIC:                             8385.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.0849      0.890      6.840      0.0